# FAF level Food Outflow Prediction GNN Model 1.0


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, GCNConv
import torch.nn.init as init
import matplotlib.pyplot as plt
from model import GCN, GAT
from utils import train, test


node_df = pd.read_csv("data/faf_features.csv")
edges_df = pd.read_csv("data/FAF5_SCTG1.csv")
distance_matrix = pd.read_csv("data/FAF_distance_matrix.csv")

In [2]:
# PCA for feature reduction
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
features_scaler = StandardScaler()
node_df.iloc[:, 1:] = features_scaler.fit_transform(node_df.iloc[:, 1:])
node_df.head()

# Select the features to be reduced
features = node_df.drop(columns=['FAF_Zone'])

# Apply PCA for feature reduction
pca = PCA(n_components=15)  # Adjust the number of components as needed
reduced_features = pca.fit_transform(features)

# Create a new DataFrame with the reduced features
reduced_df = pd.DataFrame(reduced_features, columns=[f'PC{i+1}' for i in range(reduced_features.shape[1])])
reduced_df = pd.concat([node_df['FAF_Zone'], reduced_df], axis=1)

# Update node_df with the reduced features
node_df = reduced_df

# Verify the changes
print(node_df.head())


   FAF_Zone       PC1       PC2       PC3       PC4       PC5       PC6  \
0        11 -2.006389 -1.821179 -0.404133 -0.051127 -0.039310 -0.031745   
1        12 -2.252152 -2.966382 -0.515396 -0.162503 -0.493764 -0.271856   
2        19 -0.920872 -0.456392  0.187202  1.573729  2.261417  1.221892   
3        20 -2.195409 -2.849311 -0.482327 -0.235577 -0.419569 -0.166418   
4        41 -1.771784  3.844207 -0.414339 -1.369063 -0.507529 -0.092140   

        PC7       PC8       PC9      PC10      PC11      PC12      PC13  \
0 -0.246841 -0.120840 -0.225222  0.184540 -0.209944  0.092995 -0.052166   
1 -0.260259 -0.188177 -0.436677  0.285102 -0.211547 -0.020487 -0.225732   
2  0.113730  0.256403  0.855411  0.019502 -0.085308  1.110365  0.740245   
3 -0.278788 -0.171916  0.000443  0.156871 -0.079351 -0.046955  0.267000   
4 -0.305219 -0.244953 -0.370453  0.173976 -0.438941 -0.218421 -0.102978   

       PC14      PC15  
0  0.176254  0.079381  
1  0.111855 -0.045005  
2  0.978474  0.417557  
3 

In [3]:
# Filter edges_df to only include the relevant columns
edges_df = edges_df[['dms_orig', 'dms_dest', 'tons_2017', 'dms_mode', 'dist_band']]
edges_df.head()

,dms_orig,dms_dest,tons_2017,dms_mode,dist_band
0,11,11,51.010231,1,1
1,11,19,385.622345,1,2
2,11,129,1.360447,1,3
3,11,131,12.489625,1,2
4,11,139,5.134423,1,2


In [4]:
# Group by origin and destination and sum the tons values
# Get the most common mode and distance band
edges_df = edges_df.groupby(['dms_orig', 'dms_dest']).agg({
    'tons_2017': 'sum',  # Sum the tons values
    'dms_mode': lambda x: x.mode()[0] if not x.empty else None,  # Get the most common mode
    'dist_band': lambda x: x.mode()[0] if not x.empty else None  # Get the most common distance band
}).reset_index()
edges_df.head()

,dms_orig,dms_dest,tons_2017,dms_mode,dist_band
0,11,11,51.010231,1,1
1,11,12,17.012234,1,2
2,11,19,385.622345,1,2
3,11,20,0.000000,4,8
4,11,61,0.000056,4,7


In [5]:
print(len(edges_df))

6680


In [6]:
# add missing edges with default 0 values
uniques = edges_df['dms_orig'].unique()
for i in uniques:
    for j in uniques:
        # Check if the pair (i, j) exists in the dataframe
        if not ((edges_df['dms_orig'] == i) & (edges_df['dms_dest'] == j)).any():
            # If not, add a new row with tons_2017 = 0 (which will be set to small_value later)
            # For mode and dist_band, use the most common values as defaults
            default_mode = edges_df['dms_mode'].mode()[0]
            default_dist_band = edges_df['dist_band'].mode()[0]

            new_row = pd.DataFrame({
                'dms_orig': [i],
                'dms_dest': [j],
                'tons_2017': [0],  # This will be adjusted to small_value in the next cell
                'dms_mode': [0],
                'dist_band': [0]
            })

            # Append the new row to the dataframe
            edges_df = pd.concat([edges_df, new_row], ignore_index=True)

In [7]:
# add distance between origin and destination to edges_df
distances = []
for i, row in edges_df.iterrows():

    src = row['dms_orig']
    dst = row['dms_dest']
    # Convert to string with leading zeros, ensuring they're integers first
    src_str = str(int(src))
    dst_str = str(int(dst)).zfill(3)
    distance_matrix['FAF_Zone'] = distance_matrix['FAF_Zone'].astype(str)
    dist = distance_matrix.loc[distance_matrix['FAF_Zone'] == src_str, dst_str].values[0]
    distances.append(dist)

edges_df['distance'] = distances
print(edges_df.head())


   dms_orig  dms_dest   tons_2017  dms_mode  dist_band     distance
0        11        11   51.010231         1          1     0.000000
1        11        12   17.012234         1          2   330.684950
2        11        19  385.622345         1          2    71.542974
3        11        20    0.000000         4          8  5740.453730
4        11        61    0.000056         4          7  2746.348341


In [8]:
# Create a mapping dictionary for FAF zones
faf_zones = node_df['FAF_Zone'].unique()
zone_to_idx = {zone: idx for idx, zone in enumerate(faf_zones)}
idx_to_zone = {idx: zone for idx, zone in enumerate(faf_zones)}
node_df['FAF_Zone'] = node_df['FAF_Zone'].map(zone_to_idx)

# Remap the origin and destination in edges_df
edges_df['dms_orig'] = edges_df['dms_orig'].map(zone_to_idx)
edges_df['dms_dest'] = edges_df['dms_dest'].map(zone_to_idx)

# Check the remapped data
print("\nEdges dataframe with remapped indices:")
print(edges_df.head())



Edges dataframe with remapped indices:
   dms_orig  dms_dest   tons_2017  dms_mode  dist_band     distance
0         0         0   51.010231         1          1     0.000000
1         0         1   17.012234         1          2   330.684950
2         0         2  385.622345         1          2    71.542974
3         0         3    0.000000         4          8  5740.453730
4         0         8    0.000056         4          7  2746.348341


In [9]:
#one hot encode the dms_mode and dist_band
edges_df = pd.get_dummies(edges_df, columns=['dms_mode', 'dist_band'], dtype=int)
print(edges_df.head())


   dms_orig  dms_dest   tons_2017     distance  dms_mode_0  dms_mode_1  \
0         0         0   51.010231     0.000000           0           1   
1         0         1   17.012234   330.684950           0           1   
2         0         2  385.622345    71.542974           0           1   
3         0         3    0.000000  5740.453730           0           0   
4         0         8    0.000056  2746.348341           0           0   

   dms_mode_2  dms_mode_4  dms_mode_5  dms_mode_7  dist_band_0  dist_band_1  \
0           0           0           0           0            0            1   
1           0           0           0           0            0            0   
2           0           0           0           0            0            0   
3           0           1           0           0            0            0   
4           0           1           0           0            0            0   

   dist_band_2  dist_band_3  dist_band_4  dist_band_5  dist_band_6  \
0         

In [10]:
# Transform the target columns
edges_df['tons_2017'] = np.log1p(edges_df['tons_2017'])




In [11]:
edges_df['distance'] = features_scaler.fit_transform(edges_df.iloc[:, 3].values.reshape(-1, 1))

print(edges_df.head())

   dms_orig  dms_dest  tons_2017  distance  dms_mode_0  dms_mode_1  \
0         0         0   3.951440 -1.311748           0           1   
1         0         1   2.891051 -1.073709           0           1   
2         0         2   5.957448 -1.260249           0           1   
3         0         3   0.000000  2.820443           0           0   
4         0         8   0.000056  0.665175           0           0   

   dms_mode_2  dms_mode_4  dms_mode_5  dms_mode_7  dist_band_0  dist_band_1  \
0           0           0           0           0            0            1   
1           0           0           0           0            0            0   
2           0           0           0           0            0            0   
3           0           1           0           0            0            0   
4           0           1           0           0            0            0   

   dist_band_2  dist_band_3  dist_band_4  dist_band_5  dist_band_6  \
0            0            0       

In [12]:


# Extract node features
node_features = []
for i, row in node_df.iterrows():
    node_features.append(torch.tensor(row.iloc[1:].values, dtype=torch.float))
node_features = torch.stack(node_features)
# Extract edge information (source, target)
edge_index = torch.tensor(edges_df[['dms_orig', 'dms_dest']].values.T, dtype=torch.long)

# Extract edge target values (food flow)
edge_y = torch.tensor(edges_df['tons_2017'].values, dtype=torch.float).view(-1, 1)

# Extract additional edge features
edge_attr = torch.tensor(edges_df.iloc[:, 3:].values, dtype=torch.float)  # Excluding source, target, food_flow

# Create PyG Data object
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=edge_y)


In [13]:
# Create train/test split properly
num_edges = data.edge_index.shape[1]
train_size = int(num_edges * 0.8)
indices = torch.randperm(num_edges, generator=torch.Generator().manual_seed(42))
train_indices = indices[:train_size]
test_indices = indices[train_size:]

train_data = Data(
    x=data.x,
    edge_index=data.edge_index[:, train_indices],
    edge_attr=data.edge_attr[train_indices],
    y=data.y[train_indices]
)

test_data = Data(
    x=data.x,
    edge_index=data.edge_index[:, test_indices],
    edge_attr=data.edge_attr[test_indices],
    y=data.y[test_indices]
)

# Debug info
print(f"Train data: {len(train_indices)} edges, Test data: {len(test_indices)} edges")
print(f"Train shapes - edge_index: {train_data.edge_index.shape}, y: {train_data.y.shape}")
print(f"Test shapes - edge_index: {test_data.edge_index.shape}, y: {test_data.y.shape}")


gcn_model = GCN(in_channels=data.x.shape[1], hidden_channels=64, edge_feature_dim=data.edge_attr.shape[1])
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
optimizer = torch.optim.SGD(gcn_model.parameters(), lr=0.01, momentum=0.9)
loss = nn.MSELoss()

epochs = 500
# Train the model
for epoch in range(epochs):  # Reduced epochs for faster debugging

    loss_value = train(gcn_model, train_data, loss, optimizer)


    if epoch % 100 == 0 or epoch == epochs-1:
        print(f'Epoch {epoch}, Loss: {loss_value:.7f}')
        mse, r2, accuracy = test(gcn_model, test_data)
        rmse = np.sqrt(mse)
        print(f"Test MSE: {mse:.7f}, R²: {r2:.7f}, Accuracy: {accuracy:.7f}, RMSE: {rmse:.7f}")




Train data: 13939 edges, Test data: 3485 edges
Train shapes - edge_index: torch.Size([2, 13939]), y: torch.Size([13939, 1])
Test shapes - edge_index: torch.Size([2, 3485]), y: torch.Size([3485, 1])
Epoch 0, Loss: 0.6037652
Test MSE: 0.5585471, R²: -0.1487805, Accuracy: 0.7420373, RMSE: 0.7473601
Epoch 100, Loss: 0.2724480
Test MSE: 0.2924808, R²: 0.3984460, Accuracy: 0.8748924, RMSE: 0.5408149
Epoch 200, Loss: 0.2694708
Test MSE: 0.2921169, R²: 0.3991943, Accuracy: 0.8766140, RMSE: 0.5404784
Epoch 300, Loss: 0.2680950
Test MSE: 0.2916277, R²: 0.4002005, Accuracy: 0.8769010, RMSE: 0.5400257
Epoch 400, Loss: 0.2671180
Test MSE: 0.2911862, R²: 0.4011086, Accuracy: 0.8771880, RMSE: 0.5396167
Epoch 499, Loss: 0.2662142
Test MSE: 0.2905937, R²: 0.4023273, Accuracy: 0.8748924, RMSE: 0.5390674


In [14]:
gat_model = GAT(in_channels=data.x.shape[1], hidden_channels=64, edge_feature_dim=data.edge_attr.shape[1])
# optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
optimizer = torch.optim.SGD(gat_model.parameters(), lr=0.01, momentum=0.9)
loss = nn.MSELoss()

epochs = 500
# Train the model
for epoch in range(epochs):  # Reduced epochs for faster debugging
    # Fix the train function call - it should return a tensor loss, not use the loss object
    loss_value = train(gat_model, train_data, loss, optimizer)

    # Print shapes periodically for debugging
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f'Epoch {epoch}, Loss: {loss_value:.7f}')
        mse, r2, accuracy = test(gat_model, test_data)
        rmse = np.sqrt(mse)
        print(f"Test MSE: {mse:.7f}, R²: {r2:.7f}, Accuracy: {accuracy:.7f}, RMSE: {rmse:.7f}")


Epoch 0, Loss: 0.5820481
Test MSE: 0.4983346, R²: -0.0249396, Accuracy: 0.7417504, RMSE: 0.7059282
Epoch 100, Loss: 0.3631998
Test MSE: 0.3784314, R²: 0.2216688, Accuracy: 0.8783357, RMSE: 0.6151678
Epoch 200, Loss: 0.2958981
Test MSE: 0.3126062, R²: 0.3570534, Accuracy: 0.8809182, RMSE: 0.5591120
Epoch 300, Loss: 0.2785603
Test MSE: 0.2958129, R²: 0.3915927, Accuracy: 0.8769010, RMSE: 0.5438869
Epoch 400, Loss: 0.2752846
Test MSE: 0.2936680, R²: 0.3960042, Accuracy: 0.8748924, RMSE: 0.5419115
Epoch 499, Loss: 0.2737745
Test MSE: 0.2931401, R²: 0.3970900, Accuracy: 0.8760402, RMSE: 0.5414241


In [15]:
# Save the GAT model
torch.save(gat_model.state_dict(), 'models/gat_model.pth')
print("GAT model saved to saved_models/gat_model.pth")

# Save the GCN model
torch.save(gcn_model.state_dict(), 'models/gcn_model.pth')
print("GCN model saved to saved_models/gcn_model.pth")




GAT model saved to saved_models/gat_model.pth
GCN model saved to saved_models/gcn_model.pth
